In [1]:
import itertools
import os
import sys
import copy
sys.path.append("colorization_pytorch")
sys.path.append("flownet2_pytorch")
import torch
import torch.nn as nn
from colorization_pytorch.models import create_model
from flownet2_pytorch.models import FlowNet2
import torch.nn.functional as F
import numpy as np
from matplotlib import pyplot as plt
from datasets import *
from torch.utils.data import DataLoader
from tqdm import tqdm
from colorization_pytorch.options.train_options import TrainOptions
from flownet2_pytorch.parser import initialize_args
import torchvision.transforms as transforms
from colorization_pytorch.util import util

class Mask(nn.Module):
    def __init__(self, in_channels):
        super(Mask,self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=in_channels, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels=64, out_channels=8, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.1),
            nn.Conv2d(in_channels=8, out_channels=1, kernel_size=3, stride=1, padding=1),
            nn.LeakyReLU(0.1),
        )

    def forward(self, data):
        return self.cnn(data)


class VideoColorization(nn.Module):
    def __init__(self, opt, args, batchNorm=False, div_flow=20.):
        super(VideoColorization,self).__init__()
        self.opt = opt
        self.color_model = create_model(opt)
        self.color_model.setup(opt)
        self.color_model.print_networks(False)
        self.flownet = FlowNet2(args, batchNorm=batchNorm, div_flow=div_flow)
        checkpoint = torch.load(args.resume)
        self.flownet.load_state_dict(checkpoint['state_dict'])
        #self.ori_index = torch.tensor(list(itertools.product(np.arange(H), np.arange(W)))). \
            #reshape(H, W, -1)
        self.mask = Mask(256)

    def forward(self, color_data, flow_data):
        self.color_model.set_input(color_data[0])
        _,_, previous_feature_map = self.color_model.encode()
        self.color_model.set_input(color_data[1])
        conv1_2, conv2_2, feature_map = self.color_model.encode()
        flow = self.flownet(flow_data)  # B * 2 * 384 * 1024
        #print("flow")
        FlowWeight = F.interpolate(flow, previous_feature_map.size()[-2:], mode='bilinear')
        B, C, H, W = previous_feature_map.size()
        #print(previous_feature_map.shape)
        flow_copy = FlowWeight.clone().detach().cpu().long().permute(0, 2, 3, 1)
        predicted_feature_map = torch.zeros(previous_feature_map.shape).cuda()
        for h in range(H):
            for w in range(W):
                res_h = h + flow_copy[0,h,w,0]
                res_w = w + flow_copy[0,h,w,1]
                if res_h < 0:
                    res_h = 0
                elif res_h > H-1:
                    res_h = H-1
                if res_w < 0:
                    res_w = 0
                elif res_w > W-1:
                    res_w = W-1
                predicted_feature_map[:,:,res_h,res_w] = previous_feature_map[:,:,h,w]
            
        # B*512*22*22
        #print(predicted_feature_map.shape)
        delta_feature_map = torch.abs(predicted_feature_map - feature_map)
        M = self.mask(delta_feature_map)
        #print(M.shape)
        output_feature_map = (1 - M) * feature_map + M * predicted_feature_map
        fake_B_class, fake_B_reg = self.color_model.decode(conv1_2, conv2_2, output_feature_map)
        #print(fake_B_reg.shape)
        return fake_B_reg, FlowWeight
    
    def infer_forward(self, color_data, flow_data, pre_feature_map=None):
        if pre_feature_map == None:
            self.color_model.set_input(color_data[0])
            _,_, previous_feature_map = self.color_model.encode()
        self.color_model.set_input(color_data[1])
        conv1_2, conv2_2, feature_map = self.color_model.encode()
        flow = self.flownet(flow_data)  # B * 2 * 384 * 1024
        #print("flow")
        FlowWeight = F.interpolate(flow, previous_feature_map.size()[-2:], mode='bilinear')
        B, C, H, W = previous_feature_map.size()
        #print(previous_feature_map.shape)
        flow_copy = FlowWeight.clone().detach().cpu().long().permute(0, 2, 3, 1)
        predicted_feature_map = torch.zeros(previous_feature_map.shape).cuda()
        for h in range(H):
            for w in range(W):
                res_h = h + flow_copy[0,h,w,0]
                res_w = w + flow_copy[0,h,w,1]
                if res_h < 0:
                    res_h = 0
                elif res_h > H-1:
                    res_h = H-1
                if res_w < 0:
                    res_w = 0
                elif res_w > W-1:
                    res_w = W-1
                predicted_feature_map[:,:,res_h,res_w] = previous_feature_map[:,:,h,w]
            
        # B*512*22*22
        #print(predicted_feature_map.shape)
        delta_feature_map = torch.abs(predicted_feature_map - feature_map)
        M = self.mask(delta_feature_map)
        #print(M.shape)
        output_feature_map = (1 - M) * feature_map + M * predicted_feature_map
        fake_B_class, fake_B_reg = self.color_model.decode(conv1_2, conv2_2, output_feature_map)
        #print(fake_B_reg.shape)
        return fake_B_reg, FlowWeight, output_feature_map
        

        
class flow_args():
    def __init__(self):
        self.rgb_max = 255
        self.fp16 = False
        self.fp16_scale = 1024.
        self.crop_size = [256,256]
        self.inference_size = [-1,-1]
        self.resume = './check_point/FlowNet2_checkpoint.pth.tar'
        

opt = TrainOptions().parse()
args = flow_args()
debug = VideoColorization(opt, args)


----------------- Options ---------------
                   ab_max: 110.0                         
                  ab_norm: 110.0                         
                 ab_quant: 10.0                          
             aspect_ratio: 1.0                           
           avg_loss_alpha: 0.986                         
               batch_size: 8                             
                    beta1: 0.9                           
          checkpoints_dir: ./colorization_pytorch/checkpoints
           classification: False                         
             dataset_mode: aligned                       
             display_freq: 200                           
               display_id: 1                             
            display_ncols: 5                             
             display_port: 8097                          
           display_server: http://localhost              
          display_winsize: 384                           
              epoch_count:

In [3]:
train_set = MpiSintel(args, root = "data/training/")
train_loader = DataLoader(train_set, batch_size = 8, shuffle=True, num_workers=4)

In [4]:
class solver():
    def __init__(self, model):
        self.model = model
        self.lr = 1e-4
        self.optimizer_flow = torch.optim.Adam(self.model.flownet.parameters(), lr=2e-4, weight_decay=1e-6)
        self.optimizer_color_network = []
        self.use_D = opt.lambda_GAN > 0
        self.optimizer_G = torch.optim.Adam(self.model.color_model.netG.parameters(),
                                            lr=0, betas=(opt.beta1, 0.999)) #lr=opt.lr
        self.optimizer_color_network.append(self.optimizer_G)

        if self.use_D:
            self.optimizer_D = torch.optim.Adam(self.model.color_model.netD.parameters(),
                                                    lr=0, betas=(opt.beta1, 0.999))
            self.optimizer_color_network.append(self.optimizer_D)
        
        self.optimizer_mask = torch.optim.Adam(self.model.mask.parameters(), lr=1e-4, weight_decay=1e-6)
        self.batch_size = 5
        self.epoch = 100
        self.H = 384
        self.W = 1024
        #self.ori_index = torch.tensor(list(itertools.product(np.arange(self.H), np.arange(self.W)))). \
         #   reshape(self.H, self.W, -1)
        
    def plot_grad_flow(self, named_parameters):
        '''Plots the gradients flowing through different layers in the net during training.
        Can be used for checking for possible gradient vanishing / exploding problems.
    
        Usage: Plug this function in Trainer class after loss.backwards() as 
        "plot_grad_flow(self.model.named_parameters())" to visualize the gradient flow'''
        ave_grads = []
        max_grads= []
        layers = []
        for n, p in named_parameters:
            if(p.requires_grad) and ("bias" not in n):
                layers.append(n)
                ave_grads.append(p.grad.abs().mean())
                max_grads.append(p.grad.abs().max())
        plt.bar(np.arange(len(max_grads)), max_grads, alpha=0.1, lw=1, color="c")
        plt.bar(np.arange(len(max_grads)), ave_grads, alpha=0.1, lw=1, color="b")
        plt.hlines(0, 0, len(ave_grads)+1, lw=2, color="k" )
        plt.xticks(range(0,len(ave_grads), 1), layers, rotation="vertical")
        plt.xlim(left=0, right=len(ave_grads))
        plt.ylim(bottom = -0.001, top=0.02) # zoom in on the lower gradient regions
        plt.xlabel("Layers")
        plt.ylabel("average gradient")
        plt.title("Gradient flow")
        plt.grid(True)
        plt.legend([matplotlib.lines.Line2D([0], [0], color="c", lw=4),
                matplotlib.lines.Line2D([0], [0], color="b", lw=4),
                matplotlib.lines.Line2D([0], [0], color="k", lw=4)], ['max-gradient', 'mean-gradient', 'zero-gradient'])
        plt.show()
        
    def pixel_flow(self, flow, img):
        flow_copy = flow.clone().detach().cpu().long().permute(0, 2, 3, 1)
        B, C, H, W = img.size()
        next_img = torch.zeros(img.shape).cuda()
        for h in range(H):
            for w in range(W):
                res_h = h + flow_copy[0,h,w,0]
                res_w = w + flow_copy[0,h,w,1]
                if res_h < 0:
                    res_h = 0
                elif res_h > H-1:
                    res_h = H-1
                if res_w < 0:
                    res_w = 0
                elif res_w > W-1:
                    res_w = W-1
                next_img[:,:,res_h,res_w] = img[:,:,h,w]
        return next_img
        
    def eval_metric(self, O_1, O_2, flow, mask):
        R_2 = self.pixel_flow(flow, O_1)
        Estab = torch.sum(mask * (O_2 - R_2) * (O_2 - R_2))
        return Estab
    
    def cohe_loss(self, O_2, S_1, flow, mask):
        R_2 = self.pixel_flow(flow, S_1)
        L_cohe = torch.sum(mask * (O_2 - R_2) * (O_2 - R_2))
        print (self.alpha*L_cohe)
        return L_cohe
    
    def occ_loss(self, O_2, S_2, mask):
        L_occ = torch.sum((1-mask) * (O_2 - S_2) * (O_2 - S_2))
        print (self.beta*L_occ)
        return L_occ
        
    def flow_loss(self, F_est, flow):
        F_down = F.interpolate(flow, F_est.size()[-2:], mode='bilinear')
        L_flow = torch.sum((F_est-F_down) * (F_est-F_down))
        print (self.gamma*L_flow)
        return L_flow
        
    def train(self, train_loader):
        self.alpha = 1e-6
        self.beta = 2e-7
        self.gamma = 5e-5
        self.train_records = []
        for epoch in range(1, self.epoch):
            batch = 0
            epoch_cohe_loss = 0
            epoch_occ_loss = 0
            epoch_flow_loss = 0
            epoch_total_loss = 0
            
            for Img, flow, mask in tqdm(train_loader):
                batch+=1
                Img = Img.cuda()
                flow = flow.cuda()
                mask = mask.type(torch.FloatTensor).cuda()
                #print(flow.shape)
                #print(mask.shape)
                temp_1 = util.get_colorization_data(Img[:,:,0,:,:].unsqueeze(0),self.model.opt)
                I_1 = temp_1['A'].unsqueeze(1)
                S_1 = temp_1['B']
                I_1 = I_1.repeat(1,3,1,1,1)
                temp_2 = util.get_colorization_data(Img[:,:,1,:,:].unsqueeze(0),self.model.opt)
                I_2 = temp_2['A'].unsqueeze(1)
                S_2 = temp_2['B']
                I_2 = I_2.repeat(1,3,1,1,1)
                inp = torch.cat((I_1,I_2), dim=2)
                self.optimizer_flow.zero_grad()
#                 self.optimizer_G.zero_grad()
                self.optimizer_mask.zero_grad()
                O_2, F_est = self.model([temp_1,temp_2],inp)
                #print(F_est.shape)
                
                lcohe = self.alpha*self.cohe_loss(O_2,S_1,flow,mask)
                locc = self.beta*self.occ_loss(O_2,S_2,mask)
                lflow = self.gamma*self.flow_loss(F_est,flow)
                loss = lcohe+locc+lflow
                epoch_cohe_loss+=lcohe.cpu().item()
                epoch_occ_loss+=locc.cpu().item()
                epoch_flow_loss+=lflow.cpu().item()
                epoch_total_loss+=loss.cpu().item()
                
                print(batch, loss)
                print(epoch_cohe_loss / batch, epoch_occ_loss / batch, epoch_flow_loss / batch, epoch_total_loss / batch)
                loss.backward()
                self.optimizer_flow.step()
                #self.optimizer_G.step()
                self.optimizer_mask.step()
                del Img
                del flow
                del mask
            
            print ('epoch_cohe_loss', epoch_cohe_loss/batch)
            print ('epoch_occ_loss', epoch_occ_loss/batch)
            print ('epoch_flow_loss', epoch_flow_loss/batch)
            print ('epoch_total_loss', epoch_total_loss/batch)
            self.train_records.append([epoch_cohe_loss/batch, epoch_occ_loss/batch, epoch_flow_loss/batch, epoch_total_loss/batch])
            
    
    def inference(self, video, ground_true = True):
        self.model.eval()
        video = video.cuda()
        frame_1 = video[0]
        temp_1 = util.get_colorization_data(frame_1.unsqueeze(0),self.model.opt)
        I_1 = temp_1['A'].unsqueeze(1)
        I_1 = I_1.repeat(1,3,1,1,1)
        S_1 = temp_1['B']
        res = [S_1]
        F_1 = None
        for i in range(1,len(video)):
            frame_2 = video[i]
            temp_2 = util.get_colorization_data(frame_2.unsqueeze(0),self.model.opt)
            I_2 = temp_2['A'].unsqueeze(1)
            S_2 = temp_2['B']
            I_2 = I_2.repeat(1,3,1,1,1)
            inp = torch.cat((I_1,I_2), dim=2)
            O_2, F_est, F_1 = self.model.infer_forward([temp_1,temp_2],inp, F_1)
            res.append(O_2)
            I_1 = I_2
            temp_1 = temp_2
        return res

In [ ]:
debug = debug.cuda()
solv = solver(debug)
solv.train(train_loader)

  0%|          | 0/131 [00:00<?, ?it/s]/opt/anaconda3/lib/python3.7/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/opt/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


tensor(15.2375, device='cuda:0', grad_fn=<MulBackward0>)
tensor(34.6163, device='cuda:0', grad_fn=<MulBackward0>)
tensor(21.0131, device='cuda:0', grad_fn=<MulBackward0>)
1 tensor(70.8669, device='cuda:0', grad_fn=<AddBackward0>)
15.23753833770752 34.616275787353516 21.013059616088867 70.86687469482422


  1%|          | 1/131 [00:27<59:27, 27.44s/it]

tensor(7.7415, device='cuda:0', grad_fn=<MulBackward0>)
tensor(27.2491, device='cuda:0', grad_fn=<MulBackward0>)
tensor(171.0027, device='cuda:0', grad_fn=<MulBackward0>)
2 tensor(205.9933, device='cuda:0', grad_fn=<AddBackward0>)
11.489534378051758 30.932701110839844 96.0078649520874 138.43010330200195


  2%|▏         | 2/131 [00:41<50:20, 23.41s/it]

...

  5%|▌         | 7/131 [01:51<32:06, 15.53s/it]

tensor(24.0134, device='cuda:0', grad_fn=<MulBackward0>)
tensor(17.4188, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1283.6522, device='cuda:0', grad_fn=<MulBackward0>)
8 tensor(1325.0844, device='cuda:0', grad_fn=<AddBackward0>)
7.713465094566345 22.272912740707397 287.7454149723053 317.73179626464844


  6%|▌         | 8/131 [02:05<30:47, 15.02s/it]

tensor(3.9535, device='cuda:0', grad_fn=<MulBackward0>)
tensor(22.1513, device='cuda:0', grad_fn=<MulBackward0>)
tensor(6.5578, device='cuda:0', grad_fn=<MulBackward0>)
9 tensor(32.6625, device='cuda:0', grad_fn=<AddBackward0>)
7.295690536499023 22.259396235148113 256.50234174728394 286.0574315388997


  7%|▋         | 9/131 [02:19<29:57, 14.73s/it]

In [10]:
res = solv.train_records
res

[[5.530109718441963, 19.073199218466083, 722.8373375903559, 747.4406480133988],
 [5.758796557058929, 18.977148990139707, 782.6957028298888, 807.4316489823902]]

In [11]:
solv.train(train_loader)


  0%|          | 0/131 [00:00<?, ?it/s]

tensor(2.6681, device='cuda:0', grad_fn=<MulBackward0>)
tensor(5.7897, device='cuda:0', grad_fn=<MulBackward0>)
tensor(885.6094, device='cuda:0', grad_fn=<MulBackward0>)
1 tensor(894.0672, device='cuda:0', grad_fn=<AddBackward0>)
2.668130874633789 5.789722919464111 885.609375 894.0671997070312



  1%|          | 1/131 [00:10<23:47, 10.98s/it]

tensor(4.9545, device='cuda:0', grad_fn=<MulBackward0>)
tensor(17.8664, device='cuda:0', grad_fn=<MulBackward0>)
tensor(676.7277, device='cuda:0', grad_fn=<MulBackward0>)
2 tensor(699.5486, device='cuda:0', grad_fn=<AddBackward0>)
3.8113224506378174 11.82807469367981 781.1685180664062 796.8078918457031



  2%|▏         | 2/131 [00:21<23:12, 10.79s/it]

...

tensor(5.4472, device='cuda:0', grad_fn=<MulBackward0>)
tensor(12.4508, device='cuda:0', grad_fn=<MulBackward0>)
tensor(5.0364e+23, device='cuda:0', grad_fn=<MulBackward0>)
31 tensor(5.0364e+23, device='cuda:0', grad_fn=<AddBackward0>)
5.214293543369539 19.07642322970975 1.625402254715032e+22 1.625402254715032e+22



 24%|██▎       | 31/131 [05:25<17:35, 10.55s/it]

tensor(2.0170, device='cuda:0', grad_fn=<MulBackward0>)
tensor(5.9342, device='cuda:0', grad_fn=<MulBackward0>)
tensor(1.2273e+23, device='cuda:0', grad_fn=<MulBackward0>)
32 tensor(1.2273e+23, device='cuda:0', grad_fn=<AddBackward0>)
5.114378223195672 18.665727615356445 1.9581462938161223e+22 1.9581462938161223e+22



 24%|██▍       | 32/131 [05:35<17:25, 10.56s/it]

KeyboardInterrupt: 

In [12]:
solv.train_records

[[5.234981638784627, 19.361601509665714, 707.2549589572062, 731.8515388037413],
 [5.779030986483149, 19.014239506867096, 800.8659639940917, 825.6592318010694]]